<a href="https://colab.research.google.com/github/billsioros/thesis/blob/master/Nanorough_surface_Super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✔️ Prerequisites

First of all we need to take care of a few **prerequisites**, most notably:

- Install the various pip modules that we will be using.
- Install some linux specific dependencies of our [content loss](#content-loss).
- Initialize the Random Number Generator(s), so that our experiments can be replicated.
- Determine:
  - The current working directory, as it's going to be used to reference various files such as the dataset, our model checkpoints e.t.c
  - The available hardware backend. GPU utilization is preferable, as it results in higher complition time.
- `(Optionally)` Mount Google Drive, where we can load our dataset from.

## Configuring our Loggers

In [ ]:
import logging.config
import os

LOGGING_LEVEL = os.environ.get("LOGGING_LEVEL", "CRITICAL").upper()

logging.config.dictConfig(
    {
        "version": 1,
        "disable_existing_loggers": True,
        "formatters": {
            "standard": {"format": "[%(asctime)s] %(levelname)s:%(name)s: %(message)s"}
        },
        "handlers": {
            "default": {
                "level": LOGGING_LEVEL,
                "formatter": "standard",
                "class": "logging.StreamHandler",
            }
        },
        "loggers": {"": {"handlers": ["default"], "level": LOGGING_LEVEL}},
    }
)

In [ ]:
import logging

logger = logging.getLogger()

## Determining the Current Working Directory

In [ ]:
from pathlib import Path

BASE_DIR = Path.cwd()

## Mounting Google Drive

In [ ]:
GDRIVE_DIR = BASE_DIR / "drive"

In [ ]:
try:
    from google.colab import drive

    drive.mount(f"{GDRIVE_DIR}")
except ImportError:
    pass

## Installing [graphviz](https://graphviz.org/) & [libgraphviz-dev](https://packages.debian.org/jessie/libgraphviz-dev)

The aforementioned packages are required by [PyINSECT](https://github.com/billsioros/PyINSECT/tree/implementing-HPGs) and more specifically its graph plotting methods.

In [ ]:
!sudo apt-get install graphviz libgraphviz-dev 1> /dev/null

## Installing the required `pip` modules

- [torch](https://pytorch.org/) is our machine learning framework of choice.
- [numpy](https://numpy.org/), [sympy](https://www.sympy.org/en/index.html) and [scipy](https://www.scipy.org/) are used to in the context of nanorough surface generation.
- [plotly](https://plotly.com/) (which requires [pandas](https://pandas.pydata.org/)) as well as [matplotlib](https://matplotlib.org/) are used in order to plot various graphs.

In [ ]:
WHEEL_VERSION = "1.5.0"
WHEEL_FILE = "roughml-%s-py3-none-any.whl".format(
    WHEEL_VERSION,
)
WHEEL_PATH = GDRIVE_DIR / "MyDrive" / "Thesis" / WHEEL_FILE

In [ ]:
import subprocess
import sys

pip_freeze_output = subprocess.check_output(
    [sys.executable, "-m", "pip", "freeze"]
).decode()

if "roughml" not in pip_freeze_output:
    if WHEEL_PATH.is_file():
        subprocess.check_call([sys.executable, "-m", "pip", "install", WHEEL_PATH])
    else:
        raise FileNotFoundError(WHEEL_PATH)

## Initializing (a.k.a `Seeding`) the Random Number Generator(s)

We are required to seed various random number generation engines, so that our experiments can be replicated on a later date.

In [ ]:
SEED = 1234

import os
import random

import numpy as np

In [ ]:
import torch

if SEED is not None:
    np.random.seed(SEED)
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    os.environ["PYTHONHASHSEED"] = str(SEED)

## Determining available backend

By default, we are going to be utilizing the available CPU backend, if no GPU is available.

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

In [ ]:
device = torch.device(device)

# 🙃 A naive-approach

## Instantiating the **Generator** and the **Discriminator** Networks

In [ ]:
from roughml.models import PerceptronGenerator

generator = PerceptronGenerator.from_device(device)

In [ ]:
generator

In [ ]:
from roughml.models import PerceptronDiscriminator

discriminator = PerceptronDiscriminator.from_generator(generator)

In [ ]:
discriminator

## Training

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss().to(device)

In [ ]:
from pathlib import Path

CHECKPOINT_DIR = BASE_DIR / "checkpoint"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

from roughml.content.loss import NGramGraphContentLoss
from roughml.data.transforms import To, View

In [ ]:
from roughml.training.flow import TrainingFlow
from roughml.training.manager import per_epoch

training_flow = TrainingFlow(
    training_manager={
        "benchmark": True,
        "checkpoint": {"directory": CHECKPOINT_DIR, "multiple": True},
        "train_epoch": per_epoch,
        "log_every_n": 10,
        "criterion": criterion,
        "n_epochs": 10,
        "train_ratio": 0.8,
        "optimizer": {"lr": 0.0005, "weight_decay": 0},
        "dataloader": {
            "batch_size": 256,
            "shuffle": True,
            "num_workers": 0,
        },
    },
    content_loss={
        "type": NGramGraphContentLoss,
        "cache": CHECKPOINT_DIR / "n_gram_graph_content_loss.pkl",
    },
    dataset={
        "limit": 10,
        "path": GDRIVE_DIR / "MyDrive" / "Thesis" / "Datasets" / "surfaces.zip",
        "transforms": [To(device), View(1, 128, 128)],
    },
    plot={
        "save_directory": GDRIVE_DIR / "MyDrive" / "Thesis" / "Images",
        "grayscale": {"limit": 10, "save_path_fmt": "grayscale_%s_%02d.png"},
        "surface": {"limit": 10, "save_path_fmt": "surface_%s_%02d.png"},
    },
)

In [ ]:
training_flow(generator, discriminator)

# 😎 A CNN based approach

## Instantiating the **Generator** and the **Discriminator** Networks

In [ ]:
from roughml.models import CNNGenerator

generator = CNNGenerator.from_device(device)

In [ ]:
generator

In [ ]:
from roughml.models import CNNDiscriminator

discriminator = CNNDiscriminator.from_device(device)

In [ ]:
discriminator

## Training

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss().to(device)

In [ ]:
from pathlib import Path

CHECKPOINT_DIR = BASE_DIR / "checkpoint"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

from roughml.content.loss import ArrayGraph2DContentLoss
from roughml.data.transforms import To, View

In [ ]:
from roughml.training.flow import TrainingFlow
from roughml.training.manager import per_epoch

training_flow = TrainingFlow(
    training_manager={
        "benchmark": True,
        "checkpoint": {"directory": CHECKPOINT_DIR, "multiple": True},
        "train_epoch": per_epoch,
        "log_every_n": 10,
        "criterion": criterion,
        "n_epochs": 10,
        "train_ratio": 0.8,
        "optimizer": {"lr": 0.0002, "betas": (0.5, 0.999)},
        "dataloader": {
            "batch_size": 256,
            "shuffle": True,
            "num_workers": 0,
        },
    },
    content_loss={
        "type": ArrayGraph2DContentLoss,
        "cache": CHECKPOINT_DIR / "array_graph2d_content_loss.pkl",
    },
    dataset={
        "limit": 10,
        "path": GDRIVE_DIR / "MyDrive" / "Thesis" / "Datasets" / "surfaces.zip",
        "transforms": [To(device), View(1, 128, 128)],
    },
    animation={
        "indices": [
            0,
        ],
        "save_path": Path.cwd() / "cnn_per_epoch_animation.mp4",
    },
    plot={
        "save_directory": GDRIVE_DIR / "MyDrive" / "Thesis" / "Images",
        "grayscale": {"limit": 10, "save_path_fmt": "grayscale_%s_%02d.png"},
        "surface": {"limit": 10, "save_path_fmt": "surface_%s_%02d.png"},
    },
)

In [ ]:
training_flow(generator, discriminator)

## Dismounting Google Drive and persisting any changes made

In [ ]:
drive.flush_and_unmount()